# <center> Exploring Karachi and Lahore Using Foursquare API and Clustering
## <center>Ashraf Ul Kabir</center>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2> Table of Contents</h2>  
<font size = 3>
1. <a href="#item1">Explore Dataset</a>    
2. <a href="#item2">Explore Neighborhoods in Karachi</a>   
3. <a href="#item3">Analyze Each Neighborhood</a>  
4. <a href="#item4">Cluster Neighborhoods</a>  
5. <a href="#item5">Examine Clusters</a>    
6. <a href="#item6">Proposed Further Analysis</a>
</font>
<br>

</div>

### Import necessary Libraries

In [1]:
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from scikit-learn
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library

print ('Folium installed')
print ('Libraries imported.')

Folium installed
Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

#### Read data into a *pandas* dataframe

In [2]:
karachi_data = pd.read_csv('Karachi_Neighbourhoods.csv')

# display first five rows of dataframe
karachi_data.head()

,postal code,borough,neighborhood,latitude,longitude
0,M1B,Karachi,Bahadurabad,24.882222,67.068886
1,M1C,Karachi,Dhoraji,24.888646,67.072612
2,M1E,Karachi,PECHS,24.869016,67.060942
3,M1G,Karachi,DHA Phase 1,24.842924,67.065027
4,M1H,Karachi,DHA Phase 2,24.836308,67.057689


#### Create a map of Karachi with neighborhoods superimposed on top

In [3]:

# karachi 25.083890, 67.226387
latitude = 24.8607
longitude = 67.0011

# create map of Karachi using latitude and longitude values
map_karachi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, postal_code, label_list in zip(karachi_data['latitude'], karachi_data['longitude'], karachi_data['postal code'], karachi_data['neighborhood']):
    label = str(postal_code) + ', ' + (label_list)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_karachi)  
    
map_karachi

#### Define Foursquare Credentials and Version

Make sure that you have created a Foursquare developer account and have your credentials handy

In [4]:
#CLIENT_ID = 'WSGFOUYS5Q4ACWXQ3WC0KOGD3NNDE2MQOOLXG0UPM5FRH2PJ' # your Foursquare ID
#CLIENT_SECRET = 'S4GL5U4TW1SVMBFOEHP1KY3GK15VNU2BGA5HKFRAV0ZRSOMH' # your Foursquare Secret
CLIENT_ID = 'MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ'
CLIENT_SECRET = 'RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD'
VERSION = '20171018' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

#print ()'Your credentails:'
#print 'CLIENT_ID: ' + CLIENT_ID
#print 'CLIENT_SECRET:' + CLIENT_SECRET

<a id='item2'></a>

## 2. Explore Neighborhoods in Karachi

#### Let's create a function to explore all the neighborhoods in Karachi and find out the characteristics of public checking into different locations

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&time=any&day=any&section=food'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups']
        
        url2 = 'https://api.foursquare.com/v2/venues/4e97063677c83a3315a6a530?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&time=any&day=any'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        
        for v in results[0]['items']:
            url2 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&time=any&day=any&section=food'.format(
            v['venue']['id'],
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
            results2 = requests.get(url2).json()["response"]
            print(url2)
            print (v['venue']['name'])
            
            likes=0
            
            try: 
                #print(results2['venue']['likes']['count'])
                likes=results2['venue']['likes']['count']
            
            except KeyError as e:
                print (e)
                # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name'],
                likes) ])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood latitude', 
                  'neighborhood longitude', 
                  'venue', 
                  'venue latitude', 
                  'venue longitude', 
                  'venue category',
                  'likes']
    
    return(nearby_venues)

#### Now let's run the function on each neighborhood

In [6]:
karachi_venues = getNearbyVenues(names=karachi_data.neighborhood,
                              latitudes=karachi_data.latitude,
                              longitudes=karachi_data.longitude,
                              radius=500)

Bahadurabad
https://api.foursquare.com/v2/venues/4be3d81b99feb713151b82f6?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=24.88222182,67.06888584&radius=500&limit=100&time=any&day=any&section=food
Tooso
https://api.foursquare.com/v2/venues/5011981fe4b06b8dcb028f55?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=24.88222182,67.06888584&radius=500&limit=100&time=any&day=any&section=food
14th Street Pizza Co
https://api.foursquare.com/v2/venues/4f9167b2e4b04fef008a0491?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=24.88222182,67.06888584&radius=500&limit=100&time=any&day=any&section=food
California Pizza
https://api.foursquare.com/v2/venues/4e2b05cdfa76bbf847d4a5e9?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&

https://api.foursquare.com/v2/venues/4c2092eba34cd13a2f397d74?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=24.869016000000002,67.060942&radius=500&limit=100&time=any&day=any&section=food
Ambala
https://api.foursquare.com/v2/venues/4d1f70335acaa35d27e0bf35?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=24.869016000000002,67.060942&radius=500&limit=100&time=any&day=any&section=food
Rehman Burger
DHA Phase 1
https://api.foursquare.com/v2/venues/504077f1e4b08d5b6169ad62?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=24.842924,67.065027&radius=500&limit=100&time=any&day=any&section=food
McDonald's
https://api.foursquare.com/v2/venues/4e664a9aaeb73603168bb70d?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client

https://api.foursquare.com/v2/venues/52667bc711d20866cfb3315e?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=24.913593,67.030808&radius=500&limit=100&time=any&day=any&section=food
New Alhamd Bakery
https://api.foursquare.com/v2/venues/51586e71e4b016cb46879843?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=24.913593,67.030808&radius=500&limit=100&time=any&day=any&section=food
Imperial court
North Nazimabad
https://api.foursquare.com/v2/venues/4e2e783a1fc729417e4f476b?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=24.940203,67.045163&radius=500&limit=100&time=any&day=any&section=food
Hardee's
https://api.foursquare.com/v2/venues/53f4ea5f498ee602a2dd6e2c?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_sec

#### Let's check the size of the resulting dataframe

In [7]:
print (karachi_venues.shape)
karachi_venues

(76, 8)


,neighborhood,neighborhood latitude,neighborhood longitude,venue,venue latitude,venue longitude,venue category,likes
0,Bahadurabad,24.882222,67.068886,Tooso,24.882845,67.066238,Fast Food Restaurant,44
1,Bahadurabad,24.882222,67.068886,14th Street Pizza Co,24.880810,67.068622,Pizza Place,9
2,Bahadurabad,24.882222,67.068886,California Pizza,24.882242,67.067556,Pizza Place,34
3,Bahadurabad,24.882222,67.068886,OPTP,24.882949,67.067678,Snack Place,23
4,Bahadurabad,24.882222,67.068886,Zameer Ansari,24.879285,67.071986,BBQ Joint,33
5,Bahadurabad,24.882222,67.068886,Crescent Roll Corner,24.882625,67.065822,Snack Place,6
6,Bahadurabad,24.882222,67.068886,Al Kabab,24.882076,67.068035,BBQ Joint,6
7,Bahadurabad,24.882222,67.068886,Subway,24.883871,67.064696,Sandwich Place,10
8,Bahadurabad,24.882222,67.068886,Domino's Pizza,24.882119,67.067383,Pizza Place,8
9,Bahadurabad,24.882222,67.068886,The Burger Shack,24.878725,67.066045,Burger Joint,8


#### Let's check how many venues were returned for each neighborhood

In [8]:
karachi_venues.groupby('neighborhood').mean()
#karachi_venues.groupby('neighborhood').count()

,neighborhood latitude,neighborhood longitude,venue latitude,venue longitude,likes
neighborhood,,,,,
Bahadurabad,24.882222,67.068886,24.881136,67.067598,12.611111
Clifton,24.819237,67.023010,24.816430,67.022235,83.500000
DHA Phase 1,24.842924,67.065027,24.841873,67.063733,12.000000
DHA Phase 2,24.836308,67.057689,24.835234,67.057378,0.000000
DHA Phase 4,24.823663,67.057712,24.824506,67.059646,1.400000
DHA Phase 5,24.806616,67.045208,24.807226,67.043792,1.000000
DHA Phase 6,24.804534,67.064305,24.807092,67.060810,24.714286
Dhoraji,24.888646,67.072612,24.891766,67.073641,3.666667
Gulshan e Iqbal,24.917349,67.092606,24.918251,67.092833,0.000000


#### Let's note how many unique categories can be curated from all the returned venues

In [9]:
print ('{} uniques categories'.format(len(karachi_venues['venue category'].unique())))

26 uniques categories


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [10]:
# one hot encoding
karachi_onehot = pd.get_dummies(karachi_venues[['venue category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
karachi_onehot['likes'] = karachi_venues['likes']
karachi_onehot['neighborhood'] = karachi_venues['neighborhood'] 



# move neighborhood column to the first column
fixed_columns = [karachi_onehot.columns[-1]] + list(karachi_onehot.columns[:-1])
karachi_onehot = karachi_onehot[fixed_columns]

karachi_onehot.head()

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Sandwich Place,Snack Place,Southern / Soul Food Restaurant,Steakhouse,likes
0,Bahadurabad,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,44
1,Bahadurabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,9
2,Bahadurabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,34
3,Bahadurabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,23
4,Bahadurabad,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33


And let's examine the new dataframe size

In [11]:
karachi_onehot.shape

(76, 28)

#### Next, let's group rows by neighborhood

In [12]:
karachi_grouped = karachi_onehot.groupby('neighborhood').mean().reset_index()
karachi_grouped

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Sandwich Place,Snack Place,Southern / Soul Food Restaurant,Steakhouse,likes
0,Bahadurabad,0.000000,0.055556,0.00,0.111111,0.000000,0.0,0.111111,0.000000,0.055556,0.055556,0.055556,0.000000,0.000000,0.111111,0.000000,0.0,0.000000,0.000000,0.055556,0.222222,0.000000,0.000000,0.055556,0.111111,0.00,0.000000,12.611111
1,Clifton,0.000000,0.000000,0.00,0.250000,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,83.500000
2,DHA Phase 1,0.000000,0.000000,0.00,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,12.000000
3,DHA Phase 2,0.000000,0.000000,0.00,0.000000,0.250000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
4,DHA Phase 4,0.000000,0.200000,0.00,0.000000,0.200000,0.2,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,1.400000
5,DHA Phase 5,0.000000,0.000000,0.00,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,1.000000
6,DHA Phase 6,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.142857,0.142857,0.000000,0.000000,0.142857,0.142857,0.142857,0.000000,0.00,0.000000,24.714286
7,Dhoraji,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.00,0.000000,3.666667
8,Gulshan e Iqbal,0.000000,0.000000,0.25,0.000000,0.250000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000
9,Nazimabad,0.000000,0.000000,0.00,0.000000,0.750000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,1.500000


#### Let's confirm the new size

In [13]:
karachi_grouped.shape

(12, 28)

#### Let's print each neighborhood along with the top 5 most common venues

In [14]:
num_top_venues = 5

for hood in karachi_grouped['neighborhood']:
    #print("----"+hood+"----")
    temp = karachi_grouped.loc[karachi_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.loc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = karachi_grouped['neighborhood']

for ind in np.arange(karachi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(karachi_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted

<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 7 clusters

In [17]:
# set number of clusters
kclusters = 7

karachi_grouped_clustering = karachi_grouped.drop('neighborhood', 1)
karachi_grouped_clustering = karachi_grouped_clustering.drop(columns=['likes'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(karachi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:15] 



Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [18]:
karachi_merged = karachi_grouped
#print(karachi_venues.shape)
#print (karachi_grouped.shape)

#print(karachi_merged)

karachi_merged = karachi_merged.merge(karachi_data.set_index('neighborhood'), on='neighborhood',how='left')
# add clustering labels
karachi_merged['cluster_labels'] = kmeans.labels_

# merge karachi_grouped with karachi_data to add latitude/longitude for each neighborhood
karachi_merged = karachi_merged.join(neighborhoods_venues_sorted.set_index('neighborhood'), on='neighborhood')


#karachi_merged # check the last columns!

Finally, let's visualize the resulting clusters

In [19]:
import folium
from folium.features import DivIcon

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, likes in zip(karachi_merged['latitude'], karachi_merged['longitude'], karachi_merged['neighborhood'], karachi_merged['cluster_labels'], karachi_merged['likes']):
    label = str(poi) + ' Cluster ' + str(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
for lat, lon, poi, cluster, likes in zip(karachi_merged['latitude'], karachi_merged['longitude'], karachi_merged['neighborhood'], karachi_merged['cluster_labels'], karachi_merged['likes']):
     
    folium.map.Marker(
    [lat, lon],
    icon=DivIcon(
        
        html='<div style="color:white" >%d</div>' % likes,
        )
    ).add_to(map_clusters)
    
     
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

#### Cluster 1

In [20]:
karachi_merged.loc[karachi_merged['cluster_labels'] == 0]
karachi_merged.loc[karachi_merged['cluster_labels'] == 0,'cluster_labels']='Commercial Neighborhood'
karachi_merged.loc[karachi_merged['cluster_labels'] == 'Commercial Neighborhood']

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Sandwich Place,Snack Place,Southern / Soul Food Restaurant,Steakhouse,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,DHA Phase 4,0.0,0.2,0.0,0.0,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4,M1K,Karachi,24.823663,67.057712,Commercial Neighborhood,likes,American Restaurant,Bakery,Breakfast Spot,Burger Joint,Indonesian Restaurant,Donut Shop,Asian Restaurant,BBQ Joint,Cafeteria


#### Cluster 2

In [21]:
karachi_merged.loc[karachi_merged['cluster_labels'] == 1]
karachi_merged.loc[karachi_merged['cluster_labels'] == 1,'cluster_labels']='Activity Oriented'
karachi_merged.loc[karachi_merged['cluster_labels'] == 'Activity Oriented']

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Sandwich Place,Snack Place,Southern / Soul Food Restaurant,Steakhouse,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,DHA Phase 5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,M1L,Karachi,24.806616,67.045208,Activity Oriented,likes,BBQ Joint,Pizza Place,Donut Shop,American Restaurant,Asian Restaurant,Bakery,Breakfast Spot,Burger Joint,Cafeteria


#### Cluster 3

In [22]:
karachi_merged.loc[karachi_merged['cluster_labels'] == 2]
karachi_merged.loc[karachi_merged['cluster_labels'] == 2,'cluster_labels']='Foodies Haven Neighborhood'
karachi_merged.loc[karachi_merged['cluster_labels'] == 'Foodies Haven Neighborhood']

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Sandwich Place,Snack Place,Southern / Soul Food Restaurant,Steakhouse,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bahadurabad,0.000000,0.055556,0.0,0.111111,0.000000,0.0,0.111111,0.0,0.055556,0.055556,0.055556,0.00,0.0,0.111111,0.0,0.0,0.000000,0.000000,0.055556,0.222222,0.000000,0.000000,0.055556,0.111111,0.0,0.000000,12.611111,M1B,Karachi,24.882222,67.068886,Foodies Haven Neighborhood,likes,Pizza Place,BBQ Joint,Burger Joint,Fast Food Restaurant,Snack Place,Deli / Bodega,Chinese Restaurant,Sandwich Place,Pakistani Restaurant
1,Clifton,0.000000,0.000000,0.0,0.250000,0.000000,0.0,0.000000,0.0,0.250000,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,83.500000,M1R,Karachi,24.819237,67.023010,Foodies Haven Neighborhood,likes,BBQ Joint,Pizza Place,Café,Diner,Donut Shop,American Restaurant,Asian Restaurant,Bakery,Breakfast Spot
6,DHA Phase 6,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.142857,0.142857,0.000000,0.00,0.0,0.000000,0.0,0.0,0.142857,0.142857,0.000000,0.000000,0.142857,0.142857,0.142857,0.000000,0.0,0.000000,24.714286,M1M,Karachi,24.804534,67.064305,Foodies Haven Neighborhood,likes,Sandwich Place,Restaurant,Portuguese Restaurant,Japanese Restaurant,Italian Restaurant,Café,Chinese Restaurant,Deli / Bodega,American Restaurant
10,North Nazimabad,0.076923,0.000000,0.0,0.153846,0.153846,0.0,0.000000,0.0,0.076923,0.076923,0.000000,0.00,0.0,0.076923,0.0,0.0,0.000000,0.000000,0.000000,0.153846,0.000000,0.076923,0.076923,0.000000,0.0,0.076923,15.307692,M1T,Karachi,24.940203,67.045163,Foodies Haven Neighborhood,likes,Pizza Place,BBQ Joint,Bakery,Café,Chinese Restaurant,Steakhouse,Fast Food Restaurant,Restaurant,Sandwich Place


#### Cluster 4

In [23]:
karachi_merged.loc[karachi_merged['cluster_labels'] == 3]
karachi_merged.loc[karachi_merged['cluster_labels'] == 3,'cluster_labels']='Multi-cultural Neighborhood'
karachi_merged.loc[karachi_merged['cluster_labels'] == 'Multi-cultural Neighborhood']

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Sandwich Place,Snack Place,Southern / Soul Food Restaurant,Steakhouse,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,DHA Phase 1,0.0,0.0,0.00,0.0,0.333333,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.333333,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,12.0,M1G,Karachi,24.842924,67.065027,Multi-cultural Neighborhood,likes,Indian Restaurant,Bakery,Fast Food Restaurant,Sandwich Place,Deli / Bodega,American Restaurant,Asian Restaurant,BBQ Joint,Southern / Soul Food Restaurant
3,DHA Phase 2,0.0,0.0,0.00,0.0,0.250000,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,M1H,Karachi,24.836308,67.057689,Multi-cultural Neighborhood,Fast Food Restaurant,Donut Shop,Bakery,Cafeteria,American Restaurant,Asian Restaurant,BBQ Joint,Breakfast Spot,Burger Joint,Café
8,Gulshan e Iqbal,0.0,0.0,0.25,0.0,0.250000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,M1V,Karachi,24.917349,67.092606,Multi-cultural Neighborhood,Fast Food Restaurant,Southern / Soul Food Restaurant,Asian Restaurant,Bakery,Donut Shop,American Restaurant,BBQ Joint,Breakfast Spot,Burger Joint,Cafeteria


#### Cluster 5

In [24]:
karachi_merged.loc[karachi_merged['cluster_labels'] == 4]
karachi_merged.loc[karachi_merged['cluster_labels'] == 4,'cluster_labels']='Shopping Oriented'
karachi_merged.loc[karachi_merged['cluster_labels'] == 'Shopping Oriented']

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Deli / Bodega,Diner,Donut Shop,Fast Food Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Sandwich Place,Snack Place,Southern / Soul Food Restaurant,Steakhouse,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Dhoraji,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,3.666667,M1C,Karachi,24.888646,67.072612,Shopping Oriented,likes,Snack Place,Cafeteria,Diner,Donut Shop,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot


## 6. Comparing with Lahore

It would be interesting to compare how New York Manhattan neighborhoods compare to those of Karachi's

In [25]:

lahore_data = pd.read_csv("lahore_neighbourhoods.csv")

lahore_data.head()

,postal code,borough,neighborhood,latitude,longitude
0,M1B,Lahore,Model Town,31.483879,74.322244
1,M1C,Lahore,DHA Phase 5,31.468314,74.410735
2,M1H,Lahore,Johar Town,31.464052,74.274852
3,M1L,Lahore,Wapda Town,31.435365,74.259612
4,M1M,Lahore,Cavalry Ground,31.501546,74.367835


In [26]:
lahore_data.shape # show number of (rows, columns)
#lahore 31.5204° N, 74.3587° E
latitude= 31.5204
longitude= 74.3587

## 7. Explore Neighborhoods in Lahore

#### Let's create a function to explore all the neighborhoods in Karachi and find out the characteristics of public checking into different locations

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&time=any&day=any&section=food'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups']
        
        url2 = 'https://api.foursquare.com/v2/venues/4e97063677c83a3315a6a530?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&time=any&day=any'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        
        for v in results[0]['items']:
            url2 = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&time=any&day=any&section=food'.format(
            v['venue']['id'],
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
            results2 = requests.get(url2).json()["response"]
            print(url2)
            print (v['venue']['name'])
            
            likes=0
            
            try: 
                #print(results2['venue']['likes']['count'])
                likes=results2['venue']['likes']['count']
            
            except KeyError as e:
                print (e)
                # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name'],
                likes) ])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood latitude', 
                  'neighborhood longitude', 
                  'venue', 
                  'venue latitude', 
                  'venue longitude', 
                  'venue category',
                  'likes']
    
    return(nearby_venues)

#### Now let's run the function on each neighborhood

In [28]:
lahore_venues = getNearbyVenues(names=lahore_data.neighborhood,
                              latitudes=lahore_data.latitude,
                              longitudes=lahore_data.longitude,
                              radius=500)

Model Town
https://api.foursquare.com/v2/venues/5145e731e4b086ca86d203df?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=31.483878999999998,74.322244&radius=500&limit=100&time=any&day=any&section=food
Amore Lounge
https://api.foursquare.com/v2/venues/5360ff36498e5fb4ece79193?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=31.483878999999998,74.322244&radius=500&limit=100&time=any&day=any&section=food
Level 3 Khokha
DHA Phase 5
https://api.foursquare.com/v2/venues/50672ff4e4b0b6a07d12ff37?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=31.468314000000003,74.410735&radius=500&limit=100&time=any&day=any&section=food
Jammin Java
https://api.foursquare.com/v2/venues/54398bc2498e6e2ec8775948?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3

https://api.foursquare.com/v2/venues/51dee507498e0bcc947e50ed?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=31.507939,74.350094&radius=500&limit=100&time=any&day=any&section=food
Butler's Chocolate Cafe
https://api.foursquare.com/v2/venues/4bec20aca9900f475f961840?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=31.507939,74.350094&radius=500&limit=100&time=any&day=any&section=food
Burger Hub
https://api.foursquare.com/v2/venues/4b58599af964a520e25328e3?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=31.507939,74.350094&radius=500&limit=100&time=any&day=any&section=food
Hardee's
https://api.foursquare.com/v2/venues/5371e90811d2e483cfdaa8fb?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S4

https://api.foursquare.com/v2/venues/535521fe498e317fdb1c1586?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=31.507939,74.350094&radius=500&limit=100&time=any&day=any&section=food
Mozzarella 27
https://api.foursquare.com/v2/venues/5455151d498ecdd6e4563757?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=31.507939,74.350094&radius=500&limit=100&time=any&day=any&section=food
The Lahore Social
https://api.foursquare.com/v2/venues/4d4ee5dee0f2a1cd47e9918d?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41KK1ZAPHUDEPJVDZWJVMGSMLJW45CEXOZWJYCD&v=20171018&ll=31.507939,74.350094&radius=500&limit=100&time=any&day=any&section=food
Sheikhoos
https://api.foursquare.com/v2/venues/4bfff5e492a6c928126243e2?&client_id=MLQWFSCWI1X3MXEALXORUNFILQ3BU2CV4SX05ZYHK4YQ2JNZ&client_secret=RTFSNC15S41K

#### Let's check the size of the resulting dataframe

In [29]:
print (lahore_venues.shape)
lahore_venues.head()

(67, 8)


,neighborhood,neighborhood latitude,neighborhood longitude,venue,venue latitude,venue longitude,venue category,likes
0,Model Town,31.483879,74.322244,Amore Lounge,31.480996,74.323002,Diner,1
1,Model Town,31.483879,74.322244,Level 3 Khokha,31.481670,74.319535,Cafeteria,0
2,DHA Phase 5,31.468314,74.410735,Jammin Java,31.468656,74.409841,Café,7
3,DHA Phase 5,31.468314,74.410735,Cafe Martinez,31.467537,74.413715,Bistro,0
4,DHA Phase 5,31.468314,74.410735,Lums Pdc,31.471208,74.410107,Food,3


#### Let's check how many venues were returned for each neighborhood

In [30]:
lahore_venues.groupby('neighborhood').mean()

,neighborhood latitude,neighborhood longitude,venue latitude,venue longitude,likes
neighborhood,,,,,
Anarkali,31.572836,74.309661,31.573364,74.312157,3.000000
Cavalry Ground,31.501546,74.367835,31.501416,74.367179,3.500000
DHA Phase 5,31.468314,74.410735,31.470025,74.410757,2.000000
Gulberg,31.507939,74.350094,31.507983,74.350531,24.473684
Icchra,31.535384,74.320123,31.537746,74.319713,3.500000
Johar Town,31.464052,74.274852,31.463875,74.274768,9.000000
Misri Shah,31.586986,74.334098,31.587252,74.336406,2.000000
Model Town,31.483879,74.322244,31.481333,74.321269,0.500000
Shad Bagh,31.599987,74.338327,31.600312,74.339893,2.000000


#### Let's note how many unique categories can be curated from all the returned venues

In [31]:
print ('{} uniques categories'.format(len(lahore_venues['venue category'].unique())))
print (lahore_venues['venue category'].unique())

26 uniques categories
['Diner' 'Cafeteria' 'Café' 'Bistro' 'Food' 'Chinese Restaurant'
 'Restaurant' 'Asian Restaurant' 'Pizza Place' 'Snack Place'
 'Fast Food Restaurant' 'Mediterranean Restaurant' 'Bakery' 'BBQ Joint'
 'Breakfast Spot' 'Afghan Restaurant' 'Pakistani Restaurant'
 'Fried Chicken Joint' 'Burger Joint' 'Korean Restaurant'
 'American Restaurant' 'Portuguese Restaurant' 'Donut Shop'
 'Indian Restaurant' 'Italian Restaurant' 'African Restaurant']


<a id='item3'></a>

## 8. Analyze Each Neighborhood in Lahore

In [32]:
# one hot encoding
lahore_onehot = pd.get_dummies(lahore_venues[['venue category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lahore_onehot['likes'] = lahore_venues['likes']
lahore_onehot['neighborhood'] = lahore_venues['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lahore_onehot.columns[-1]] + list(lahore_onehot.columns[:-1])
lahore_onehot = lahore_onehot[fixed_columns]

lahore_onehot.head()

,neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Diner,Donut Shop,Fast Food Restaurant,Food,Fried Chicken Joint,Indian Restaurant,Italian Restaurant,Korean Restaurant,Mediterranean Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Snack Place,likes
0,Model Town,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Model Town,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,DHA Phase 5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7
3,DHA Phase 5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,DHA Phase 5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3


And let's examine the new dataframe size

In [33]:
lahore_onehot.shape

(67, 28)

#### Next, let's group rows by neighborhood

In [34]:
lahore_grouped = lahore_onehot.groupby('neighborhood').mean().reset_index()
lahore_grouped

,neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Diner,Donut Shop,Fast Food Restaurant,Food,Fried Chicken Joint,Indian Restaurant,Italian Restaurant,Korean Restaurant,Mediterranean Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Snack Place,likes
0,Anarkali,0.0,0.000000,0.000000,0.200000,0.000000,0.200000,0.0,0.200000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.200000,0.000000,0.000000,0.2,0.00,3.000000
1,Cavalry Ground,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.250000,0.0,0.00,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.25,3.500000
2,DHA Phase 5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.0,0.200000,0.200000,0.0,0.000000,0.000000,0.2,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.2,0.00,2.000000
3,Gulberg,0.0,0.026316,0.052632,0.078947,0.026316,0.052632,0.0,0.000000,0.052632,0.0,0.289474,0.026316,0.0,0.026316,0.131579,0.0,0.00,0.052632,0.026316,0.026316,0.00,0.078947,0.026316,0.026316,0.0,0.00,24.473684
4,Icchra,0.0,0.000000,0.000000,0.250000,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.0,0.00,3.500000
5,Johar Town,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.500000,0.000000,0.0,0.25,9.000000
6,Misri Shah,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.500000,0.000000,0.000000,0.0,0.00,2.000000
7,Model Town,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.000000,0.5,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.500000
8,Shad Bagh,0.0,0.000000,0.000000,0.333333,0.333333,0.000000,0.0,0.333333,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,2.000000


#### Let's confirm the new size

In [35]:
lahore_grouped.shape

(9, 28)

#### Let's print each neighborhood along with the top 5 most common venues

In [36]:
num_top_venues = 5

for hood in lahore_grouped['neighborhood']:
    print("----"+hood+"----")
    temp = lahore_grouped.loc[lahore_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.loc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anarkali----
              venue  freq
0             likes   3.0
1  Asian Restaurant   0.2
2        Restaurant   0.2
3            Bakery   0.2
4    Breakfast Spot   0.2


----Cavalry Ground----
                      venue  freq
0                     likes  3.50
1      Fast Food Restaurant  0.25
2               Snack Place  0.25
3                    Bakery  0.25
4  Mediterranean Restaurant  0.25


----DHA Phase 5----
        venue  freq
0       likes   2.0
1        Food   0.2
2  Restaurant   0.2
3      Bistro   0.2
4        Café   0.2


----Gulberg----
                  venue   freq
0                 likes  24.47
1                  Café   0.29
2  Fast Food Restaurant   0.13
3      Asian Restaurant   0.08
4  Pakistani Restaurant   0.08


----Icchra----
                  venue  freq
0                 likes  3.50
1      Asian Restaurant  0.25
2                Bakery  0.25
3  Pakistani Restaurant  0.25
4   Fried Chicken Joint  0.25


----Johar Town----
                  venue  freq
0   

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = lahore_grouped['neighborhood']

for ind in np.arange(lahore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lahore_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.shape
neighborhoods_venues_sorted


,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anarkali,likes,Restaurant,Pakistani Restaurant,Asian Restaurant,Bakery,Breakfast Spot,Chinese Restaurant,African Restaurant,American Restaurant,BBQ Joint
1,Cavalry Ground,likes,Mediterranean Restaurant,Bakery,Fast Food Restaurant,Snack Place,Chinese Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint
2,DHA Phase 5,likes,Chinese Restaurant,Restaurant,Bistro,Food,Café,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint
3,Gulberg,likes,Café,Fast Food Restaurant,Asian Restaurant,Pakistani Restaurant,American Restaurant,Bakery,Burger Joint,Indian Restaurant,African Restaurant
4,Icchra,likes,Asian Restaurant,Pakistani Restaurant,Bakery,Fried Chicken Joint,Chinese Restaurant,African Restaurant,American Restaurant,BBQ Joint,Bistro
5,Johar Town,likes,Pizza Place,Asian Restaurant,Snack Place,Chinese Restaurant,African Restaurant,American Restaurant,BBQ Joint,Bakery,Bistro
6,Misri Shah,likes,Pakistani Restaurant,Afghan Restaurant,Portuguese Restaurant,Pizza Place,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery
7,Model Town,likes,Diner,Cafeteria,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot
8,Shad Bagh,likes,Asian Restaurant,BBQ Joint,Breakfast Spot,Diner,African Restaurant,American Restaurant,Bakery,Bistro,Burger Joint


<a id='item4'></a>

## 9. Cluster Neighborhoods in Lahore

Run *k*-means to cluster the neighborhood into 5 clusters

In [49]:
# set number of clusters
kclusters = 5

lahore_grouped_clustering = lahore_grouped.drop('neighborhood', 1)
lahore_grouped_clustering = lahore_grouped_clustering.drop(columns=['likes'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lahore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15] 

array([0, 1, 1, 1, 0, 4, 3, 2, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [54]:
lahore_merged = lahore_grouped
print(lahore_data.shape)
print (lahore_grouped.shape)

lahore_merged = lahore_merged.merge(lahore_data.set_index('neighborhood'), on='neighborhood',how='left')


# add clustering labels
lahore_merged['cluster_labels'] = kmeans.labels_

# merge lahore_grouped with lahore_data to add latitude/longitude for each neighborhood
lahore_merged = lahore_merged.join(neighborhoods_venues_sorted.set_index('neighborhood'), on='neighborhood')

lahore_merged.head() # check the last columns!

(10, 5)
(9, 28)


,neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Diner,Donut Shop,Fast Food Restaurant,Food,Fried Chicken Joint,Indian Restaurant,Italian Restaurant,Korean Restaurant,Mediterranean Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Snack Place,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anarkali,0.0,0.000000,0.000000,0.200000,0.000000,0.200000,0.0,0.2,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.200000,0.000000,0.000000,0.2,0.00,3.000000,M1T,Lahore,31.572836,74.309661,0,likes,Restaurant,Pakistani Restaurant,Asian Restaurant,Bakery,Breakfast Spot,Chinese Restaurant,African Restaurant,American Restaurant,BBQ Joint
1,Cavalry Ground,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.250000,0.0,0.00,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.25,3.500000,M1M,Lahore,31.501546,74.367835,1,likes,Mediterranean Restaurant,Bakery,Fast Food Restaurant,Snack Place,Chinese Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint
2,DHA Phase 5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.0,0.000000,0.0,0.200000,0.200000,0.0,0.000000,0.000000,0.2,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.2,0.00,2.000000,M1C,Lahore,31.468314,74.410735,1,likes,Chinese Restaurant,Restaurant,Bistro,Food,Café,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint
3,Gulberg,0.0,0.026316,0.052632,0.078947,0.026316,0.052632,0.0,0.0,0.052632,0.0,0.289474,0.026316,0.0,0.026316,0.131579,0.0,0.00,0.052632,0.026316,0.026316,0.00,0.078947,0.026316,0.026316,0.0,0.00,24.473684,M1W,Lahore,31.507939,74.350094,1,likes,Café,Fast Food Restaurant,Asian Restaurant,Pakistani Restaurant,American Restaurant,Bakery,Burger Joint,Indian Restaurant,African Restaurant
4,Icchra,0.0,0.000000,0.000000,0.250000,0.000000,0.250000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.0,0.00,3.500000,M1V,Lahore,31.535384,74.320123,0,likes,Asian Restaurant,Pakistani Restaurant,Bakery,Fried Chicken Joint,Chinese Restaurant,African Restaurant,American Restaurant,BBQ Joint,Bistro


Finally, let's visualize the resulting clusters

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lahore_merged['latitude'], lahore_merged['longitude'], lahore_merged['neighborhood'], lahore_merged['cluster_labels']):
    label = str(poi) + ' Cluster ' + str(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
for lat, lon, poi, cluster, likes in zip(lahore_merged['latitude'], lahore_merged['longitude'], lahore_merged['neighborhood'], lahore_merged['cluster_labels'], lahore_merged['likes']):
     
    folium.map.Marker(
    [lat, lon],
    icon=DivIcon(
        
        html='<div style="color:white" >%d</div>' % likes,
        )
    ).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 10. Examine Clusters

#### Cluster 1

In [42]:
lahore_merged.loc[lahore_merged['cluster_labels'] == 0]
#lahore_merged.loc[lahore_merged['cluster_labels'] == 0,'cluster_labels']='Shopping & Eateries'
#lahore_merged.loc[lahore_merged['cluster_labels'] == 'Shopping & Eateries']

,neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Diner,Donut Shop,Fast Food Restaurant,Food,Fried Chicken Joint,Indian Restaurant,Italian Restaurant,Korean Restaurant,Mediterranean Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Snack Place,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anarkali,0.0,0.0,0.0,0.200000,0.000000,0.20,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.2,0.0,3.0,M1T,Lahore,31.572836,74.309661,0,likes,Restaurant,Pakistani Restaurant,Asian Restaurant,Bakery,Breakfast Spot,Chinese Restaurant,African Restaurant,American Restaurant,BBQ Joint
4,Icchra,0.0,0.0,0.0,0.250000,0.000000,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,3.5,M1V,Lahore,31.535384,74.320123,0,likes,Asian Restaurant,Pakistani Restaurant,Bakery,Fried Chicken Joint,Chinese Restaurant,African Restaurant,American Restaurant,BBQ Joint,Bistro
8,Shad Bagh,0.0,0.0,0.0,0.333333,0.333333,0.00,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,2.0,M1R,Lahore,31.599987,74.338327,0,likes,Asian Restaurant,BBQ Joint,Breakfast Spot,Diner,African Restaurant,American Restaurant,Bakery,Bistro,Burger Joint


#### Cluster 2

In [43]:
lahore_merged.loc[lahore_merged['cluster_labels'] == 1]
#lahore_merged.loc[lahore_merged['cluster_labels'] == 1,'cluster_labels']='Well-Connected and Reachable'
#lahore_merged.loc[lahore_merged['cluster_labels'] == 'Well-Connected and Reachable']

,neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Diner,Donut Shop,Fast Food Restaurant,Food,Fried Chicken Joint,Indian Restaurant,Italian Restaurant,Korean Restaurant,Mediterranean Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Snack Place,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Cavalry Ground,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.250000,0.0,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.25,3.500000,M1M,Lahore,31.501546,74.367835,1,likes,Mediterranean Restaurant,Bakery,Fast Food Restaurant,Snack Place,Chinese Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint
2,DHA Phase 5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.0,0.000000,0.0,0.200000,0.200000,0.0,0.000000,0.000000,0.2,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.2,0.00,2.000000,M1C,Lahore,31.468314,74.410735,1,likes,Chinese Restaurant,Restaurant,Bistro,Food,Café,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint
3,Gulberg,0.0,0.026316,0.052632,0.078947,0.026316,0.052632,0.0,0.0,0.052632,0.0,0.289474,0.026316,0.0,0.026316,0.131579,0.0,0.0,0.052632,0.026316,0.026316,0.00,0.078947,0.026316,0.026316,0.0,0.00,24.473684,M1W,Lahore,31.507939,74.350094,1,likes,Café,Fast Food Restaurant,Asian Restaurant,Pakistani Restaurant,American Restaurant,Bakery,Burger Joint,Indian Restaurant,African Restaurant


#### Cluster 3

In [44]:
lahore_merged.loc[lahore_merged['cluster_labels'] == 2]
#lahore_merged.loc[lahore_merged['cluster_labels'] == 2,'cluster_labels']='Hip Zone'
#lahore_merged.loc[lahore_merged['cluster_labels'] == 'Hip Zone']

,neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Diner,Donut Shop,Fast Food Restaurant,Food,Fried Chicken Joint,Indian Restaurant,Italian Restaurant,Korean Restaurant,Mediterranean Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Snack Place,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Model Town,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,M1B,Lahore,31.483879,74.322244,2,likes,Diner,Cafeteria,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot


#### Cluster 4

In [45]:
lahore_merged.loc[lahore_merged['cluster_labels'] == 3]
#lahore_merged.loc[lahore_merged['cluster_labels'] == 3,'cluster_labels']='Touristic Neighborhood'
#lahore_merged.loc[lahore_merged['cluster_labels'] == 'Touristic Neighborhood']

,neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Diner,Donut Shop,Fast Food Restaurant,Food,Fried Chicken Joint,Indian Restaurant,Italian Restaurant,Korean Restaurant,Mediterranean Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Snack Place,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Misri Shah,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,2.0,M1S,Lahore,31.586986,74.334098,3,likes,Pakistani Restaurant,Afghan Restaurant,Portuguese Restaurant,Pizza Place,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery


#### Cluster 5

In [46]:
lahore_merged.loc[lahore_merged['cluster_labels'] == 4]
#lahore_merged.loc[lahore_merged['cluster_labels'] == 4,'cluster_labels']='Specialty Markets'
#lahore_merged.loc[lahore_merged['cluster_labels'] == 'Specialty Markets']

,neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Bistro,Breakfast Spot,Burger Joint,Cafeteria,Café,Chinese Restaurant,Diner,Donut Shop,Fast Food Restaurant,Food,Fried Chicken Joint,Indian Restaurant,Italian Restaurant,Korean Restaurant,Mediterranean Restaurant,Pakistani Restaurant,Pizza Place,Portuguese Restaurant,Restaurant,Snack Place,likes,postal code,borough,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Johar Town,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.25,9.0,M1H,Lahore,31.464052,74.274852,4,likes,Pizza Place,Asian Restaurant,Snack Place,Chinese Restaurant,African Restaurant,American Restaurant,BBQ Joint,Bakery,Bistro


## 11. Conclusion

We can compare the 2 cities to evaluate their similarities and dissimilarities. 

### Similarities and Dissimilarities

Just from the onset we can understand the behaviors of the nationals living within these cities, for instance the most common venue is an eatery for both the venues.

> Most common activity among the inhabitants is eating out.
> Lahore is more touristic than Karachi
> Karachi being a coastal city has more activity based hangouts
> Karachi is more multicultural


In order to compare between the options available to Aldente, we could investigate the likability of each neighborhood, this is a cumulative mean score of all the likes for each location. Once these are collected, we can plot it on the map to label each neighborhood by the number of average likes. This basically gives an idea of the potential in that particular location. 

Upon investigating, we can see that in Karachi the cluster 3 is more lucrative and stands out. There are 4 neighborhoods namely Bahadurabad, Clifton, DHA Phase 6 and North Nazimabad. Of these choices, Clifton stands out as being way ahead. This gives us a view that it is more visited by people who are savvy in technology adoption and express it using the mobile.

Similarly, when we analyze Lahore we can determine that of all the clusters, cluster 2 having DHA Phase 5, Cavalry Ground and Gulberg are most prominent in terms of likability. Of these Gulberg appears to be the trendy place for foodies, having a diverse set of restaurants, high number of likes and densely commercialized neighborhood.

The analysis of public domain information such as Foursquare can provide an insightful lense into the public behavior and allow a new entrant to determine for itself the best option for opening a restaurant.